In [ ]:
## 导入数据

import pandas as pd
import numpy as np

ptu15 = pd.read_csv('PTU_R06_15.csv')
vis15 = pd.read_csv('VIS_R06_15.csv')
wind15 = pd.read_csv('WIND_R06_15.csv')

ptu12 = pd.read_csv('PTU_R06_12.csv')
vis12 = pd.read_csv('VIS_R06_12.csv')
wind12 = pd.read_csv('WIND_R06_12.csv')


# 修改ptu15列名
ptu15.columns = ['CREATEDATE', 'LOCALDATE (BEIJING)', 'SITE', 'PAINS',
       'QNH AERODROME', 'ST', 'QFE R06', 'ST.1', 'QFE R24',
       'ST.2', 'QFF AERODROME', 'TREND', 'TENDENCY', 'TEMP',
       'RH', 'DEWPOINT', 'TU DATA']

ptu15_select = ptu15[['CREATEDATE','LOCALDATE (BEIJING)','SITE','PAINS','QFE R06','QNH AERODROME','TEMP','RH','DEWPOINT']]

ptu12.columns = ['CREATEDATE', 'LOCALDATE (BEIJING)', 'SITE', 'PAINS',
       'QNH AERODROME', 'ST', 'QFE R06', 'ST.1', 'QFE R24',
       'ST.2', 'QFF AERODROME', 'TREND', 'TENDENCY', 'TEMP',
       'RH', 'DEWPOINT', 'TU DATA']
ptu12_select = ptu12[['CREATEDATE','LOCALDATE (BEIJING)','SITE','PAINS','QFE R06','QNH AERODROME','TEMP','RH','DEWPOINT']]





# 对VIS、WIND表同时间戳数据取平均
vis15_mean = vis15[['CREATEDATE','LOCALDATE (BEIJING)','SITE','RVR_1A','MOR_1A','LIGHTS']].groupby(by = ['CREATEDATE','LOCALDATE (BEIJING)','SITE']).mean().reset_index()
vis12_mean = vis12[['CREATEDATE','LOCALDATE (BEIJING)','SITE','RVR_1A','MOR_1A','LIGHTS']].groupby(by = ['CREATEDATE','LOCALDATE (BEIJING)','SITE']).mean().reset_index()
wind15_mean = wind15[['CREATEDATE','LOCALDATE (BEIJING)','SITE','WS2A (MPS)','WD2A','CW2A (MPS)']].groupby(by = ['CREATEDATE','LOCALDATE (BEIJING)','SITE']).mean().reset_index()
wind12_mean = wind12[['CREATEDATE','LOCALDATE (BEIJING)','SITE','WS2A (MPS)','WD2A','CW2A (MPS)']].groupby(by = ['CREATEDATE','LOCALDATE (BEIJING)','SITE']).mean().reset_index()

#并对可见度进行合成：VIS
vis15_mean['VIS']= np.where(vis15_mean['RVR_1A'] <=2000, vis15_mean['RVR_1A'], vis15_mean['MOR_1A'])
vis12_mean['VIS']= np.where(vis12_mean['RVR_1A'] <=2000, vis12_mean['RVR_1A'], vis12_mean['MOR_1A'])


data15 = ptu15_select.merge(vis15_mean,on = ['CREATEDATE','LOCALDATE (BEIJING)','SITE'] , how = 'inner')

data15 = data15.merge(wind15_mean,on = ['CREATEDATE','LOCALDATE (BEIJING)','SITE'] , how = 'inner')


data12 = ptu12_select.merge(vis12_mean,on = ['CREATEDATE','LOCALDATE (BEIJING)','SITE'] , how = 'inner')

data12 = data12.merge(wind12_mean,on = ['CREATEDATE','LOCALDATE (BEIJING)','SITE'] , how = 'inner')

# 重新保存
df1215 = data15.copy()
df1215.columns = ['CREATEDATE', 'LOCALDATE', 'SITE', 'PAINS', 'QFE',
       'QNH', 'TEMP', 'RH', 'DEWPOINT', 'RVR_1A', 'MOR_1A', 'LIGHTS',
       'VIS', 'WS2A', 'WD2A', 'CW2A']
# #删除CW2A为0的行
# df1215 = df1215[df1215['CW2A'] != 0]
# df1215.to_csv('df1215.csv')

# 重新保存
df0312 = data12.copy()
df0312.columns = ['CREATEDATE', 'LOCALDATE', 'SITE', 'PAINS', 'QFE',
       'QNH', 'TEMP', 'RH', 'DEWPOINT', 'RVR_1A', 'MOR_1A', 'LIGHTS',
       'VIS', 'WS2A', 'WD2A', 'CW2A']
# #删除CW2A为0的行
# df0312 = df0312[df0312['CW2A'] != 0]
# df0312.to_csv('df0312.csv')

## 初步数据可视化

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] # 步骤一（替换sans-serif字体）
plt.rcParams['axes.unicode_minus'] = False   # 步骤二（解决坐标轴负数的负号显示问题）
import seaborn as sns
import matplotlib.pyplot as plt 

#灯光、可见度、时间
plt.figure(figsize=(20,10))
plt.plot(np.arange(len(df1215['CREATEDATE'])),df1215['LIGHTS'],'b',label="LIGHTS")
plt.plot(np.arange(len(df1215['CREATEDATE'])),df1215['VIS'],'r',label="VIS")
plt.legend(loc="upper right") #显示图中的标签
# plt.xlabel("the number of sales")
# plt.ylabel('value of sales')
plt.title('20191215')
plt.show()

#灯光、可见度、时间
plt.figure(figsize=(20,10))
plt.plot(np.arange(len(df0312['CREATEDATE'])),df0312['LIGHTS'],'b',label="LIGHTS")
plt.plot(np.arange(len(df0312['CREATEDATE'])),df0312['VIS'],'r',label="VIS")
plt.legend(loc="upper right") #显示图中的标签
# plt.xlabel("the number of sales")
# plt.ylabel('value of sales')
plt.title('20200312')
plt.show()

## 世界时间17:00-23:59 与00:00-17:00 分段处理
---
直观看来了LIGHTS和可见度没有什么关系

df,day = df1215.copy(),'2019/12/15'
idx1 = df[df['CREATEDATE'] == day+ ' 6:00'].index
idx2 = df[df['CREATEDATE'] == day +' 17:00'].index
print(idx1,idx2)

def dayCut(df):
    dfd = df.iloc[360:1021,:]
    return dfd

df1d = dayCut(df1215)
df2d = dayCut(df0312)

## 将两天白天的数据合并

df = pd.concat([df1d,df2d])

## 数据预处理
---
RVR和MOR分别为两种定义下的能见度，其中RVR≤2000米，MOR≤10000米

def dataPre(data15):
    #去重 和去空值
    data15.dropna(inplace = True)
    data15.drop_duplicates(inplace = True)


    #去极值
    def filter_extreme_3sigma(series,n=3): #3 sigma
        mean = series.mean()
        std = series.std()
        max_range = mean + n*std
        min_range = mean - n*std
    #     print(max_range)
    #     print(min_range)
        return np.clip(series,min_range,max_range)
    for col in data15.columns:
        if isinstance(data15[col].iloc[0],str):
    #         print(col)
            pass
        else:
            data15[col] = filter_extreme_3sigma(data15[col])


    #标准化
    from sklearn.preprocessing import StandardScaler

## 标准化(使特征数据方差为1,均值为0)

    # 使用sklearn的包
    scaler = StandardScaler()
    data_notstr = data15.iloc[:,3:].copy()
    scaler.fit(data_notstr)                               # 使用transfrom必须要用fit语句
    data_notstr = scaler.transform(data_notstr)          # transfrom通过找中心和缩放等实现标准化
#     fit_trans_data_2 = scaler.fit_transform(data_2)  # fit_transfrom为先拟合数据,然后转化它将其转化为标准形式
    data15.iloc[:,3:] = data_notstr
#     print('使用fit,transform标准化的数据:\n', data15)
#     print('使用fit_transform标准化的数据:\n', fit_trans_data_2)
    return data15


# temp = df1215[(df1215.VIS!=5000) & (df1215.VIS!=6000) & (df1215.VIS!=7000) ].copy()
df_pro = dataPre(df)


## 相关性分析

df_pro.columns = ['CREATEDATE',
 'LOCALDATE',
 'SITE',
 'PAINS',
 'QFE',
 'QNH',
 'TEMP',
 'RH',
 'DEWPOINT',
 'WS2A',
 'WD2A',
 'CW2A',
 'RVR_1A',
 'MOR_1A',
 'LIGHTS',
 'VIS']

aa = df_pro.corr()
aa.apply(lambda x: x.round(2))

plt.figure(figsize=(20, 15))
sns.set(font_scale=2) # font size 2
sns.heatmap(df_pro.corr(), annot=True)
plt.savefig("./结果/heatmap.png")

plt.figure(figsize=(20, 15))
sns.set(font_scale=2) # font size 2
sns.pairplot(df_pro[[
 'PAINS',
 'QFE',
 'QNH',
 'TEMP',
 'RH',
 'DEWPOINT',
 'WS2A',
 'WD2A',
 'CW2A',
 'VIS']],kind='reg')
plt.savefig("./结果/pairplot.png")
plt.show()

## 多元线性回归

import seaborn as sns
import matplotlib.pyplot as plt 
data = df_pro.copy()
# visualize the relationship between the features and the response using scatterplots
feature_cols = ['PAINS','TEMP','RH','WS2A'] #'PAINS',
flag_col = 'VIS'

sns.set(font_scale=2) # font size 2
sns.pairplot(data, x_vars=feature_cols[:2], y_vars='VIS', height=10, aspect=0.8, kind='reg')
plt.savefig('./结果/关键变量相关性1.png')
plt.show()#注意必须加上这一句，否则无法显示。


sns.set(font_scale=2) # font size 2
sns.pairplot(data, x_vars=feature_cols[2:], y_vars='VIS', height=10, aspect=0.8, kind='reg')
plt.savefig('./结果/关键变量相关性2.png')
plt.show()#注意必须加上这一句，否则无法显示。


#seaborn的pairplot函数绘制X的每一维度和对应Y的散点图。通过设置size和aspect参数来调节显示的大小和比例。
#可以从图中看出，TV特征和销量是有比较强的线性关系的，而Radio和Sales线性关系弱一些，Newspaper和Sales线性关系更弱。
#通过加入一个参数kind='reg'，seaborn可以添加一条最佳拟合直线和95%的置信带。

#create a python list of feature names
# equivalent command to do this in one line
X = data[feature_cols]
y = data[flag_col]

##构造训练集和测试集
#default split is 75% for training and 25% for testing
from sklearn.model_selection import train_test_split  #这里是引用了交叉验证
X_train,X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
model=linreg.fit(X_train, y_train)
linreg.fit(X_train, y_train)
print(model)
print(linreg.intercept_)
print(linreg.coef_)

# pair the feature names with the coefficients
print(flag_col+' = ',end = '')
for i,j in zip(feature_cols, linreg.coef_):
    print(f'({j.round(4)}) * {i} + ',end = '')
print(f'({linreg.intercept_})')
    

---
评价回归问题
    (1)平均绝对误差(Mean Absolute Error, MAE)

    (2)均方误差(Mean Squared Error, MSE)

    (3)均方根误差(Root Mean Squared Error, RMSE)

# #模型评估

# print(type(y_pred),type(y_test))
# print (len(y_pred),len(y_test))
# print (y_pred.shape,y_test.shape)
# from sklearn import metrics
# import numpy as np
# sum_mean=0
# for i in range(len(y_pred)):
#     sum_mean+=(y_pred[i]-y_test.values[i])**2
# sum_erro=np.sqrt(sum_mean/50)
# # calculate RMSE by hand
# print("RMSE by hand:",sum_erro)
#RMSE越小越好
r_sq = model.score(X_test, y_test)
print('TEST')
print('coefficient of determination(𝑅²) :', r_sq)

y_pred = model.predict(X_test)
from sklearn import metrics
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print('MSE:',MSE)
print('RMSE:',RMSE)

r_sq = model.score(X_train, y_train)
print('TRAIN')
print('coefficient of determination(𝑅²) :', r_sq)
y_train_pred = model.predict(X_train)
from sklearn import metrics
MSE = metrics.mean_squared_error(y_train, y_train_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))

print('MSE:',MSE)
print('RMSE:',RMSE)

# 训练集绘图
plt.figure(figsize=(20,10))
x = np.arange(len(X_train))
y_train_hat = model.predict(X_train)
plt.scatter(x,y_train)
plt.plot(x, y_train_hat)
plt.show()

font = {'weight' : 'normal',
'size' : 30,
}

plt.rcParams['font.sans-serif']=['SimHei']#黑体
# 测试集绘图
y_pred = linreg.predict(X_test)
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))

plt.plot(range(len(y_pred)),y_pred,'b',label="预测值")
plt.plot(range(len(y_pred)),y_test,'r',label="真实值")
plt.legend(loc="upper right",prop = font) #显示图中的标签
# plt.title('测试集对比图')
plt.ylabel("能见度",font)
plt.xlabel('时间点',font)
plt.savefig('./结果/测试集对比.png')
plt.show()


#绘制对比曲线
X1 = X.iloc[:661,:].copy()
y1 = y.iloc[:661].copy()
plt.figure(figsize=(20,10))
x = np.arange(len(X1))
y_hat = model.predict(X1)
plt.scatter(x,y1,label = '真实值')
plt.plot(x, y_hat,label = '预测值')
plt.legend(loc="upper right",prop = font) #显示图中的标签
# plt.title('2019/12/15 能见度真实值与预测值对比图')
plt.ylabel("能见度",font)
plt.xlabel('时间点',font)
plt.savefig('./结果/第一天对比.png')
plt.show()


X1 = X.iloc[661:,:].copy()
y1 = y.iloc[661:].copy()
plt.figure(figsize=(20,10))
x = np.arange(len(X1))
y_hat = model.predict(X1)
plt.scatter(x,y1,label = '真实值')
plt.plot(x, y_hat,label = '预测值')
plt.legend(loc="upper right",prop = font) #显示图中的标签
# plt.title('2020/03/12 能见度真实值与预测值对比图')
plt.ylabel("能见度",font)
plt.xlabel('时间点',font)
plt.savefig('./结果/第二天对比.png')
plt.show()


## 模型调优

plt.figure(figsize=(20,10))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()], [y_test.min(),y_test.max()], 'k--')
# plt.title('训练集能见度')
font1= font = {'weight' : 'normal',
'size' : 30,
}
plt.xlabel('真实值',font1)
plt.ylabel('预测值',font1)
plt.savefig('./结果/测试集VIS.png')
plt.show()

# ---
#     应该去除异常值<pr>
#     可以看出VIS = 5000,6000，7000的时候有较多异常值，因此去除这些点


# # 去除VIS = 5000,6000，7000的异常值
# drop_index = y[(y.values==5000) | (y.values==6000) | (y.values==7000)  ].index.values
# X = X.drop(drop_index)
# y = y.drop(drop_index)
# # 这里划分训练集和测试集的参数random_state都是1，表示随机分配的数据是同一组
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
# # 对训练集进行训练
# lr = LinearRegression()
# lr.fit(X_train, y_train)
# # 对测试集进行预测
# y_pred = lr.predict(X_test)
# MSE = metrics.mean_squared_error(y_test, y_pred)
# RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

# print('MSE:',MSE)
# print('RMSE:',RMSE)

# plt.scatter(y_test, y_pred)
# plt.plot([y_test.min(),y_test.max()], [y_test.min(),y_test.max()], 'k--')
# plt.title('MODIFY1')
# plt.xlabel('REAL')
# plt.ylabel('PRED')